In [1]:
#5. Evaluation Report
from surprise import accuracy

def evaluate_model(predictions):
    """Calculate evaluation metrics"""
    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)
    return {'RMSE': rmse, 'MAE': mae}

def compute_top_n_metrics(testset, user_id, top_n=10):
    """Calculate precision/recall metrics"""
    test_df = pd.DataFrame([(pred.uid, pred.iid, pred.r_ui, pred.est) for pred in testset],
                         columns=['userId', 'movieId', 'actual', 'predicted'])
    user_df = test_df[test_df['userId'] == user_id]

    if user_df.empty:
        return None

    actual_liked = user_df[user_df['actual'] >= 4]['movieId'].values
    predicted_top = user_df.sort_values('predicted', ascending=False)['movieId'].head(top_n).values

    true_positives = len(np.intersect1d(actual_liked, predicted_top))
    precision = true_positives / top_n if top_n else 0
    recall = true_positives / len(actual_liked) if len(actual_liked) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'actual_count': len(actual_liked),
        'recommended_count': len(predicted_top)
    }